In [285]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 전역 설정
from matplotlib import rc
import os
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.size"] = 15

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [286]:
# 경로 설정
data_path = '../../data'
saved_path = './saved'
output_path = './submission'

In [287]:
# 데이터 불러오기 
history_df = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')
# history_df['profile_id'] = history_df['profile_id'].astype(str)
watch_e_df = pd.read_csv(os.path.join(data_path, 'watch_e_data.csv'), encoding='utf-8')
# watch_e_df['profile_id'] = watch_e_df['profile_id'].astype(str)
buy_df = pd.read_csv(os.path.join(data_path, 'buy_data.csv'), encoding='utf-8')
search_df  = pd.read_csv(os.path.join(data_path, 'search_data.csv'), encoding='utf-8')

meta = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')
meta_p = pd.read_csv(os.path.join(data_path, 'meta_data_plus.csv'), encoding='utf-8')

profile_df = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')

sample_submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'), encoding='utf-8')

In [288]:
pr_interest_keyword_dict = {'P01':'과학기술',
                           'P02':'정서/사회성',
                           'P03':'자연탐구',
                           'P04':'바른생활/안전',
                           'P05':'활동/운동',
                           'P06':'음악예술',
                           'P07':'언어논리',
                           'P08':'수리논리'}
ch_interest_keyword_dict = {'K01':'노래/율동',
                           'K02':'동물/식물',
                           'K03':'동화',
                           'K04':'만들기',
                           'K05':'숫자/계산',
                           'K06':'외국어',
                           'K07':'친구/사람',
                           'K08':'탈것/기계',
                           'K09':'활동/운동'}

pr_list = ['pr_interest_keyword_cd_1','pr_interest_keyword_cd_2','pr_interest_keyword_cd_3']
ch_list = ['ch_interest_keyword_cd_1','ch_interest_keyword_cd_2','ch_interest_keyword_cd_3']
for pr in pr_list:
    profile_df[pr] = profile_df[pr].apply(lambda x: pr_interest_keyword_dict.get(x, np.NaN))
for ch in ch_list:
    profile_df[ch] = profile_df[ch].apply(lambda x: ch_interest_keyword_dict.get(x, np.NaN))
    

In [289]:
# 태그 코드는 떨어트릴 거지만 그래도 같이 변경해주자.

meta_p.replace({'keyword_name' : '안데르센상'}, '안데르센',inplace=True)
meta_p.replace({'keyword_name' : '용감함 4'}, '용감함',inplace=True)
meta_p.replace({'keyword_name' : '의사소통'}, '의사소통(Communication)',inplace=True)
meta_p.replace({'keyword_name' : '자신감'}, '자신감(Confidence)',inplace=True)
meta_p.replace({'keyword_name' : '장선혜'}, '장선혜(작화)',inplace=True)
meta_p.replace({'keyword_name' : '최숙희'}, '최숙희(작화)',inplace=True)
meta_p.replace({'keyword_name' : '캐리'}, '캐리TV',inplace=True)

meta_p.replace({'keyword_type' : '05010a01'}, '0501090d',inplace=True)
meta_p.replace({'keyword_type' : '0509071c'}, '05090708',inplace=True)
meta_p.replace({'keyword_type' : '05030908'}, '050c1002',inplace=True)
meta_p.replace({'keyword_type' : '05070607'}, '050c1006',inplace=True)
meta_p.replace({'keyword_type' : '05010907'}, '05010914',inplace=True)
meta_p.replace({'keyword_type' : '05010909'}, '05010915',inplace=True)
meta_p.replace({'keyword_type' : '05080409'}, '0501080a',inplace=True)

In [290]:
# sub_title 리스트 담기
meta_df_sub_title_list_dict= {}

for idx, row in meta.iterrows():
    if meta_df_sub_title_list_dict.get(row['album_id'], None) == None:
        meta_df_sub_title_list_dict[row['album_id']] = [row['sub_title']]
    elif row['sub_title'] not in meta_df_sub_title_list_dict[row['album_id']]:
        meta_df_sub_title_list_dict[row['album_id']] = meta_df_sub_title_list_dict[row['album_id']]+[row['sub_title']]
        
print('전체 컨텐츠 개수:',len(set(meta.album_id)),'딕셔너리 담긴 아이템 개수:',len(meta_df_sub_title_list_dict))        

meta_df_prepro_sub_title = meta.copy()
meta_df_prepro_sub_title['sub_title'] = meta_df_prepro_sub_title['album_id'].apply(lambda x: meta_df_sub_title_list_dict.get(x, None))

# 전체 중복 삭제하기
meta_df_prepro_sub_title = meta_df_prepro_sub_title.drop_duplicates(subset=['album_id'], ignore_index=True)

print('전체 데이터수와 album_id 개수 동일 확인:', len(meta_df_prepro_sub_title), len(meta_df_prepro_sub_title.album_id.unique()))

# 메타 플러스 데이터 keyword(keyword_name, type)리스트 담기
meta_plus_type_name = meta_p.copy()

type_to_name = {}

for idx, row in meta_p.iterrows():
    type_to_name[row['keyword_type']] = row['keyword_name']
    
meta_plus_type_name['keyword_name'] = meta_plus_type_name['keyword_type'].apply(lambda x: type_to_name[x])    

name_to_type = {}
for idx, row in meta_p.iterrows():
    name_to_type[row['keyword_name']] = row['keyword_type']
    
meta_plus_type_name['keyword_type'] = meta_plus_type_name['keyword_name'].apply(lambda x: name_to_type[x])
# print('keyword_name, keyword_type 개수 확인:',len(set(meta_plus_type_name.keyword_name)), len(set(meta_plus_type_name.keyword_type)))
# print('앨범 당 키워드 중복 여부 확인:',len(meta_p),len(meta_p.drop_duplicates(subset=['album_id','keyword_type'])))

meta_plus_df_prepro_keyword = meta_plus_type_name.copy()

# meta plus data type name 동일하게 만들기

meta_df_keyword_name_list_dict= {}
meta_df_keyword_type_list_dict= {}

for idx, row in meta_plus_df_prepro_keyword.iterrows():
    if meta_df_keyword_name_list_dict.get(row['album_id'], None) == None:
        meta_df_keyword_name_list_dict[row['album_id']] = [row['keyword_name']]
    elif row['keyword_name'] not in meta_df_keyword_name_list_dict[row['album_id']]:
        meta_df_keyword_name_list_dict[row['album_id']] = meta_df_keyword_name_list_dict[row['album_id']]+[row['keyword_name']]

for idx, row in meta_plus_df_prepro_keyword.iterrows():
    if meta_df_keyword_type_list_dict.get(row['album_id'], None) == None:
        meta_df_keyword_type_list_dict[row['album_id']] = [row['keyword_type']]
    elif row['keyword_type'] not in meta_df_keyword_type_list_dict[row['album_id']]:
        meta_df_keyword_type_list_dict[row['album_id']] = meta_df_keyword_type_list_dict[row['album_id']]+[row['keyword_type']]        

meta_plus_df_prepro_keyword['keyword_name'] = meta_plus_df_prepro_keyword['album_id'].apply(lambda x: meta_df_keyword_name_list_dict.get(x, None))
meta_plus_df_prepro_keyword['keyword_type'] = meta_plus_df_prepro_keyword['album_id'].apply(lambda x: meta_df_keyword_type_list_dict.get(x, None))

meta_plus_df_keyword = meta_plus_df_prepro_keyword.drop_duplicates(subset=['album_id'],ignore_index=True).drop(columns=['keyword_value'])

# meta data merge
all_meta_df = pd.merge(meta_df_prepro_sub_title, meta_plus_df_keyword, how='left',on='album_id')
all_meta_df.info()

전체 컨텐츠 개수: 39875 딕셔너리 담긴 아이템 개수: 39875
전체 데이터수와 album_id 개수 동일 확인: 39875 39875
<class 'pandas.core.frame.DataFrame'>
Int64Index: 39875 entries, 0 to 39874
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   album_id      39875 non-null  int64  
 1   title         39875 non-null  object 
 2   sub_title     39875 non-null  object 
 3   genre_large   39875 non-null  object 
 4   genre_mid     39875 non-null  object 
 5   genre_small   12279 non-null  object 
 6   country       31773 non-null  object 
 7   run_time      39875 non-null  int64  
 8   onair_date    5208 non-null   float64
 9   cast_1        26309 non-null  object 
 10  cast_2        21217 non-null  object 
 11  cast_3        15885 non-null  object 
 12  cast_4        11980 non-null  object 
 13  cast_5        6146 non-null   object 
 14  cast_6        2540 non-null   object 
 15  cast_7        743 non-null    object 
 16  keyword_type  39875 non-null  obj

In [291]:
history_df = history_df.drop_duplicates(subset=['profile_id', 'album_id', 'log_time']).reset_index(drop = True)
history_df = history_df.sort_values(by='log_time')
# print('같은 시간 중복 시청 전처리 전 후:',len(history_meta_profile_df),len(history_meta_profile_duplicate_df))
watch_e_df = watch_e_df.drop_duplicates(subset=['profile_id', 'album_id', 'log_time']).reset_index(drop = True)

In [292]:
import datetime as dt
history_df['week_day'] = history_df['ss_id'].apply(lambda x: dt.datetime.strptime(str(x)[:8], '%Y%m%d'))
history_df['week'] = history_df['week_day'].apply(lambda x: x.isocalendar()[1])
history_df['day'] = history_df['week_day'].apply(lambda x: x.isocalendar()[2])
history_df = history_df.drop(columns='week_day')

In [293]:
print('user 수:',len(set(history_df.profile_id)), 'item 수:',len(set(history_df.album_id)))

user 수: 8311 item 수: 20695


In [294]:
all_meta_df_small = all_meta_df[['album_id','title','sub_title','genre_large','genre_mid','run_time','cast_1','cast_2']]
dataset = pd.merge(history_df, all_meta_df_small, how='left', on='album_id')
drop_columns = ['act_target_dtl','payment','short_trailer','continuous_play']
dataset = dataset.drop(columns=drop_columns)

# user tracking

In [295]:
dataset.profile_id.value_counts().describe()

count    8311.000000
mean      108.200217
std       261.375965
min         1.000000
25%        10.000000
50%        36.000000
75%       101.000000
max      8264.000000
Name: profile_id, dtype: float64

In [296]:
# 한 세션의 시청 횟수 ss_cnt 집계해서 넣어주기
ss_cnt = dataset.groupby(['profile_id']).count()['ss_id'].reset_index().rename(columns={'ss_id':'ss_cnt'})
ss_per_cnt = dataset.groupby(['profile_id','ss_id']).count()['log_time'].reset_index().rename(columns={'log_time':'ss_per_cnt'})
dataset= pd.merge(dataset, ss_cnt, how='left', on=['profile_id'])
dataset= pd.merge(dataset, ss_per_cnt, how='left', on=['profile_id','ss_id'])

- median으로 36회 이상 시청 유저 tracking

In [297]:
# median
dataset.profile_id.value_counts()[dataset.profile_id.value_counts()==36].head()

22126    36
31272    36
12042    36
16761    36
7957     36
Name: profile_id, dtype: int64

In [298]:
# mean
dataset.profile_id.value_counts()[dataset.profile_id.value_counts()==108].head()

28117    108
12603    108
9842     108
9369     108
3035     108
Name: profile_id, dtype: int64

# 1. 22126 user tracking(median)

In [299]:
user_22126 = dataset[dataset['profile_id']==22126]
profile_df[profile_df['profile_id']==22126]

,profile_id,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
6032,22126,F,7,음악예술,언어논리,수리논리,만들기,노래/율동,동화


In [300]:
# 총 4번 틈
# 책 앨범만 봄
user_22126

,profile_id,ss_id,log_time,album_id,week,day,title,sub_title,genre_large,genre_mid,run_time,cast_1,cast_2,ss_cnt,ss_per_cnt
137056,22126,20220309194614,20220309194841,4605,10,3,어떻게 먹을까,[4-5세],키즈,책,245,NaN,NaN,36,14
137145,22126,20220309194614,20220309195156,2892,10,3,모모와 토토,[4-5세],키즈,책,305,NaN,NaN,36,14
137281,22126,20220309194614,20220309195618,3016,10,3,커다란 방귀,[4-5세],키즈,책,346,NaN,NaN,36,14
137409,22126,20220309194614,20220309200054,1884,10,3,모르는척 공주,[4-5세],키즈,책,532,NaN,NaN,36,14
137563,22126,20220309194614,20220309200656,4005,10,3,첫눈을 기다리는 코딱지 코지,[4-5세],키즈,책,534,NaN,NaN,36,14
137737,22126,20220309194614,20220309201515,2184,10,3,진정한 일곱 살,[6-7세],키즈,책,269,NaN,NaN,36,14
137820,22126,20220309194614,20220309201931,4005,10,3,첫눈을 기다리는 코딱지 코지,[4-5세],키즈,책,534,NaN,NaN,36,14
137825,22126,20220309194614,20220309201947,2865,10,3,"응가하자, 끙끙",[3세이하],키즈,책,231,NaN,NaN,36,14
137909,22126,20220309194614,20220309202341,2864,10,3,과일과 채소로 만든 맛있는 그림책,"[3세이하, [하단]책 지면1]",키즈,책,233,NaN,NaN,36,14
137923,22126,20220309194614,20220309202405,2189,10,3,케이크 파티,[3세이하],키즈,책,500,NaN,NaN,36,14


In [301]:
user_22126.album_id.value_counts()

2865    3
424     3
2348    2
4005    2
2189    2
4076    2
380     2
2532    2
2533    2
2482    1
7067    1
3817    1
2867    1
2192    1
949     1
473     1
4605    1
440     1
2892    1
8292    1
2864    1
2184    1
1884    1
3016    1
7633    1
Name: album_id, dtype: int64

# 2. user 28117

In [313]:
user_28117 = dataset[dataset['profile_id']==28117]
profile_df[profile_df['profile_id']==28117]

,profile_id,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
7452,28117,M,4,활동/운동,자연탐구,바른생활/안전,동물/식물,탈것/기계,노래/율동


In [305]:
# 앨범번호가 순차적으로 재생이 많이 됨
# 이는 연속 재생시 다음 앨범아이디로 넘어가는 게 아닐까 생각됨
# 6036 ~ 6039
# 7777 ~ 7779
# 3365 ~ 3368

# 수학 컨텐츠길래 부모님이 일정 주기로 틀어주는 건 줄알았더니
# 하루 왠종일 보네 33번을 이건 아이가 좋아서 보는 듯
# 수학 컨텐츠라고 할지라도 아이가 좋아할 수도 있다..
user_28117

,profile_id,ss_id,log_time,album_id,week,day,title,sub_title,genre_large,genre_mid,run_time,cast_1,cast_2,ss_cnt,ss_per_cnt
598302,28117,20220407211223,20220407211308,125,14,4,응가공주,[4-5세],키즈,책,520,NaN,NaN,108,2
598517,28117,20220407211223,20220407212152,127,14,4,인사,[4-5세],키즈,책,438,NaN,NaN,108,2
633087,28117,20220409212159,20220409212237,339,14,6,잘했어 쌍둥이 장갑,[4-5세],키즈,책,755,NaN,NaN,108,7
633522,28117,20220409212159,20220409213836,124,14,6,고릴라 코딱지,[4-5세],키즈,책,382,NaN,NaN,108,7
633699,28117,20220409212159,20220409214502,65,14,6,치카왕자,[4-5세],키즈,책,536,NaN,NaN,108,7
633932,28117,20220409212159,20220409215402,125,14,6,응가공주,[4-5세],키즈,책,520,NaN,NaN,108,7
634118,28117,20220409212159,20220409220246,127,14,6,인사,[4-5세],키즈,책,438,NaN,NaN,108,7
634250,28117,20220409212159,20220409221008,2029,14,6,파랑오리,[4-5세],키즈,책,448,NaN,NaN,108,7
634402,28117,20220409212159,20220409221750,2029,14,6,파랑오리,[4-5세],키즈,책,448,NaN,NaN,108,7
667062,28117,20220411212003,20220411212039,2052,15,1,안녕송,"[[상단] 신나는 노래율동, 엄마까투리 율동 동요]",키즈,노래율동,60,엄마 까투리,꽁지,108,2


In [306]:
# 연속된 앨범일 경우 sub_title 같음 시리즈물인 경우 앨범아이디가 연속되는 경우가 만을 듯
all_meta_df[all_meta_df['album_id'].isin([album_id for album_id in range(6036,6040)])]
all_meta_df[all_meta_df['album_id'].isin([album_id for album_id in range(7777,7780)])]
all_meta_df[all_meta_df['album_id'].isin([album_id for album_id in range(3365,3369)])]

,album_id,title,sub_title,genre_large,genre_mid,genre_small,country,run_time,onair_date,cast_1,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7,keyword_type,keyword_name
32263,3365,세 아기 고양이,[리틀베이비범 동물 동요1],키즈,노래율동,영어,영국,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010401, 05010402, 05010403, 050c0101, 050c0...","[3세, 4세, 5세, 영어유치원, 동식물과 자연, 허락 요청하기, 충고하기, 의복..."
32264,3366,다섯 아기 오리,[리틀베이비범 동물 동요1],키즈,노래율동,영어,영국,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010401, 05010402, 05010407, 05010408, 05090...","[3세, 4세, 1세, 2세, 인지, 영어유치원, 동식물과 자연, 취미, 여가활동,..."
32265,3367,아기 오리 여섯 마리,[리틀베이비범 동물 동요1],키즈,노래율동,영어,영국,113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010401, 05010402, 05010407, 05010408, 05090...","[3세, 4세, 1세, 2세, 협동성, 리더십, 영어유치원, 동식물과 자연, 성격,..."
32266,3368,다섯 마리 알록달록 개구리,[리틀베이비범 동물 동요1],키즈,노래율동,영어,영국,149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010401, 05010402, 05010403, 05090801, 050c0...","[3세, 4세, 5세, 인지, 영어유치원, 동식물과 자연, 색깔, 수량, 동물, 음..."


In [307]:
user_28117.album_id.value_counts().head()

1738    8
2308    7
1832    7
2221    7
1737    7
Name: album_id, dtype: int64

In [308]:
# 전부 [3세이하, 그래그래 수학놀이] 타이틀이였음
# 아 그러면 부모님이 학습시키려고 틀어주는 거겠군?
all_meta_df[all_meta_df['album_id'].isin(user_28117.album_id.value_counts().head().index)]

,album_id,title,sub_title,genre_large,genre_mid,genre_small,country,run_time,onair_date,cast_1,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7,keyword_type,keyword_name
38680,1737,순서대로 찰칵!,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,NaN,한국,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010101, 05010104, 05010202, 05010401, 05010...","[시리즈, 책TV, 2분 이상 10분 미만, 3세, 4세, 5세, 엔터, 노래, 음..."
38681,2221,내 방에 놀러 와!,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,NaN,한국,133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010101, 05010104, 05010202, 05010401, 05010...","[시리즈, 책TV, 2분 이상 10분 미만, 3세, 4세, 5세, 엔터, 노래, 음..."
38682,1738,도시락을 냠냠!,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,NaN,한국,157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010101, 05010104, 05010202, 05010401, 05010...","[시리즈, 책TV, 2분 이상 10분 미만, 3세, 4세, 5세, 엔터, 노래, 음..."
38683,1832,짧게 길게 놀아 볼까,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,NaN,한국,185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010101, 05010104, 05010202, 05010401, 05010...","[시리즈, 책TV, 2분 이상 10분 미만, 3세, 4세, 5세, 엔터, 노래, 음..."
38684,2308,차례차례 나와라 얍!,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,NaN,한국,140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010101, 05010104, 05010202, 05010401, 05010...","[시리즈, 책TV, 2분 이상 10분 미만, 3세, 4세, 5세, 엔터, 노래, 음..."


In [309]:
# 수학 컨텐츠길래 부모님이 일정 주기로 틀어주는 건 줄알았더니
# 하루 왠종일 보네 33번을 이건 아이가 좋아서 보는 듯
# 수학 컨텐츠라고 할지라도 아이가 좋아할 수도 있다..
user_28117[user_28117['album_id'].isin(user_28117.album_id.value_counts().head().index)]

,profile_id,ss_id,log_time,album_id,week,day,title,sub_title,genre_large,genre_mid,run_time,cast_1,cast_2,ss_cnt,ss_per_cnt
778214,28117,20220419153203,20220419154313,1737,16,2,순서대로 찰칵!,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,135,NaN,NaN,108,8
778232,28117,20220419153203,20220419154531,2221,16,2,내 방에 놀러 와!,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,133,NaN,NaN,108,8
778250,28117,20220419153203,20220419154747,1738,16,2,도시락을 냠냠!,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,157,NaN,NaN,108,8
789850,28117,20220420132441,20220420132515,1738,16,3,도시락을 냠냠!,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,157,NaN,NaN,108,33
789880,28117,20220420132441,20220420132901,1832,16,3,짧게 길게 놀아 볼까,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,185,NaN,NaN,108,33
789901,28117,20220420132441,20220420133209,2308,16,3,차례차례 나와라 얍!,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,140,NaN,NaN,108,33
789925,28117,20220420132441,20220420133444,1737,16,3,순서대로 찰칵!,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,135,NaN,NaN,108,33
789948,28117,20220420132441,20220420133702,2221,16,3,내 방에 놀러 와!,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,133,NaN,NaN,108,33
789965,28117,20220420132441,20220420133918,1738,16,3,도시락을 냠냠!,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,157,NaN,NaN,108,33
789989,28117,20220420132441,20220420134158,1832,16,3,짧게 길게 놀아 볼까,"[3세이하, 그래그래 수학놀이]",키즈,노래율동,185,NaN,NaN,108,33


# 3. 12603 user

In [314]:

user_12603 = dataset[dataset['profile_id']==12603]
profile_df[profile_df['profile_id']==12603]

,profile_id,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
3554,12603,F,8,음악예술,NaN,NaN,숫자/계산,만들기,동물/식물


In [315]:
user_12603

,profile_id,ss_id,log_time,album_id,week,day,title,sub_title,genre_large,genre_mid,run_time,cast_1,cast_2,ss_cnt,ss_per_cnt
607250,12603,20220408173115,20220408173147,766,14,5,동물 발자국 맞추기,[마샤와 곰1],키즈,TV만화,411,마샤,곰,108,3
607315,12603,20220408173115,20220408173601,767,14,5,마술사가 된 마샤,[마샤와 곰1],키즈,TV만화,394,마샤,곰,108,3
607325,12603,20220408173115,20220408173624,481,14,5,"곰아, 뭐해?",[마샤와 곰1],키즈,TV만화,376,마샤,곰,108,3
607589,12603,20220408175038,20220408175237,6263,14,5,악당이 나타났다,"[캐리 앤 하우스 1, 캐리앤 하우스1]",키즈,놀이교실,236,꼬마캐리,꼬마캐빈,108,7
607641,12603,20220408175038,20220408175649,17807,14,5,무엇이 똑같을까?,[딩딩코딩],키즈,창의학습,355,지니,강이,108,7
607737,12603,20220408175038,20220408180153,10410,14,5,뭘까? 뭘까? 초성퀴즈,[딩딩코딩2],키즈,창의학습,373,지니,강이,108,7
607842,12603,20220408175038,20220408180810,3999,14,5,무엇이 들어있을까?,[딩딩코딩2],키즈,창의학습,350,지니,강이,108,7
607950,12603,20220408175038,20220408181407,11039,14,5,과자 길을 찾아라,[딩딩코딩2],키즈,창의학습,383,지니,강이,108,7
608099,12603,20220408175038,20220408182035,11038,14,5,빙글빙글 돌아라,[딩딩코딩2],키즈,창의학습,375,지니,강이,108,7
608265,12603,20220408175038,20220408182655,4000,14,5,달라진 인형의 집을 찾아라,[딩딩코딩2],키즈,창의학습,386,지니,강이,108,7
